# Generate Stock Time Series Data

In [26]:
import numpy as np
import pandas as pd
import bokeh.plotting as bk
# hover pan, zoom, reset, lables, etc tools
from bokeh.models import HoverTool, Label, BoxZoomTool, PanTool, ZoomInTool, ZoomOutTool, ResetTool
import requests

API_URL = 'https://api.iextrading.com/1.0'

# Make request to API and get data

In [2]:
res = requests.get(f'{API_URL}/stock/MSFT/chart/5y')

In [3]:
data = res.json()

In [4]:
df = pd.DataFrame(data)

In [5]:
df.sample(2)

change  changeOverTime  changePercent    close        date     high  \
1130  0.198373        2.343000          0.215  92.2932  2018-03-02  92.3924   
660  -0.757302        0.906081         -1.419  52.6229  2016-04-20  53.4843   

           label      low     open  unadjustedVolume    volume     vwap  
1130   Mar 2, 18  90.1210  90.8351          32830389  32830389  91.3534  
660   Apr 20, 16  52.5235  53.2855          36195714  36195714  52.9537

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1259 entries, 0 to 1258
Data columns (total 12 columns):
change              1259 non-null float64
changeOverTime      1259 non-null float64
changePercent       1259 non-null float64
close               1259 non-null float64
date                1259 non-null object
high                1259 non-null float64
label               1259 non-null object
low                 1259 non-null float64
open                1259 non-null float64
unadjustedVolume    1259 non-null int64
volume              1259 non-null int64
vwap                1259 non-null float64
dtypes: float64(8), int64(2), object(2)
memory usage: 118.1+ KB


# Candlestick Chart Setup

In [7]:
seqs = np.arange(df.shape[0])
df['seqs'] = pd.Series(seqs)

### CALCULATE THE PERCENTAGE OF CHANGE

In [8]:
df['changePercent'] = df['changePercent'].apply(lambda x: str(x) +'%')

In [9]:
# axis of one means go down the comumns, not the whole row. Default of 0 operates on the rows.
df['mid'] = df.apply(lambda x: (x['open'] + x['close'])/2, axis=1)
df.head(1)

change  changeOverTime changePercent    close        date     high  \
0  0.035355             0.0        0.128%  27.6079  2013-09-05  27.7891   

       label     low     open  unadjustedVolume    volume     vwap  seqs  \
0  Sep 5, 13  27.356  27.4886          71623991  71623991  27.6561     0   

        mid  
0  27.54825

In [10]:
df['height'] = df.apply(
    lambda x: x['close'] - x['open'] if x['close'] != x['open'] else 0.001, 
    axis=1)

In [11]:
df.sample(2)

change  changeOverTime changePercent    close        date     high  \
425  1.009000        0.626143        2.299%  44.8944  2015-05-14  44.9866   
613 -0.018795        0.691635        -0.04%  46.7025  2016-02-11  47.0972   

          label      low     open  unadjustedVolume    volume     vwap  seqs  \
425  May 14, 15  44.2586  44.2587          32980892  32980892  44.8063   425   
613  Feb 11, 16  45.5934  45.7532          48878571  48878571  46.3695   613   

          mid  height  
425  44.57655  0.6357  
613  46.22785  0.9493

In [12]:
inc = df.close > df.open
dec = df.close < df.open
w = .3

In [13]:
sourceInc = bk.ColumnDataSource(df.loc[inc])
sourceDec = bk.ColumnDataSource(df.loc[dec])

### Define type of tools we are going to use

In [14]:
hover = HoverTool(
    tooltips=[
        ('date', '@date'),
        ('low', '@low'),
        ('high', '@high'),
        ('open', '@open'),
        ('close', '@close'),
        ('percent', '@percent'),
    ]
)

In [29]:
TOOLS = [hover, BoxZoomTool(), PanTool(), ZoomInTool(), ZoomOutTool(), ResetTool()]

### Present the data

In [16]:
p = bk.figure(plot_width=1500, plot_height=800, tools=TOOLS, title='Microsoft', toolbar_location='above')

### Put the data into the chart

In [17]:
p.xaxis.major_label_orientation = np.pi/4
p.grid.grid_line_alpha=w

In [18]:
descriptor = Label(x=70, y=70, text='Label goes here')
# adds label onto the chart
p.add_layout(descriptor)

In [19]:
# set up your outer tails, and then the rectangles that go inside of the tails
p.segment(df.seqs[inc], df.high[inc], df.seqs[inc], df.low[inc], color='green')

bokeh.models.renderers.GlyphRenderer(
    id='98b94bde-2467-4335-bdf3-96be716451e6',
    data_source=bokeh.models.sources.ColumnDataSource(
        id='a3922ffe-1d10-4428-a8d8-c3e491195b6e',
        callback=None,
        data={'y1': {'__ndarray__': 'QmDl0CJbO0DByqFFtpM7QPCnxks3GTxAcT0K16PAPECX/5B++7o8QKMjufyHBD1AbAn5oGdzPUBg5dAi27k8QPrt68A5ozxAWKg1zTvOPEDoaiv2lw09QCzUmuYd5zxAyeU/pN9uPUBbsb/snmw9QJ7vp8ZLtz1AS1mGONZlPUC+nxov3cQ9QCSX/5B+2z1A63O1FfuLPkBUdCSX/2A+QA4tsp3vVz5A0NVW7C/bPkCpE9BE2FA/QOtztRX7az9AWYY41sWdP0BGtvP91GBAQI2XbhKDgEBAguLHmLtOQECI9NvXgatAQJkqGJXUaUBA5fIf0m+XQEAdOGdEaZ9AQCgPC7WmsUBA1zTvOEXXQEC0yHa+n/JAQOm3rwPn9EBAtvP91Hj5QEAu/yH99hVBQIV80LNZRUBAhjjWxW2kP0CGWtO84xxAQPCFyVTBUEBAE2HD0yuNQEBYObTIdm5AQPs6cM6IkkBA1lbsL7sfQECfPCzUmtY/QBB6Nqs+1z5Ar5RliGPtP0BCYOXQIitAQJ88LNSa1j9Af/s6cM74P0BuowG8BSJAQLprCfmgR0BAm+Ydp+jIP0BiodY07whAQA1xrIvbKEBAmSoYldRpQEBO0ZFc/ptAQB04Z0Rpn0BAHThnRGmfQECoxks3ibFAQBBYObTIxkBAI0p7gy/8QECkcD0K17NAQPKwUGua90BAB/AWSFAUQUAdyeU/pP9AQM4ZUdob7EBA+THmriX0QECI9NvXgdNAQPkx5q4l9EBAWvW52oolQUCmm8QgsJpBQIv9Zffk4UFAodY07zjNQUBHcvkP6R9CQFtCPujZbEJAEqW9wReWQUAjSnuDL+RBQCzUmuYdf0FAa5p3nKJzQUC+nxov3YRBQDsBTYQN50FAkX77OnDeQUDT3uALk6FBQKOSOgFN/EFAAiuHFtlGQUCWsgxxrHtBQKhXyjLEyUFAGy/dJAYBQkBnRGlv8L1BQPmgZ7Pq00FAPzVeukn0QUDV52or9gdCQJ7vp8ZLL0JAfGEyVTACQkBxPQrXo0BCQEtZhjjWdUJAS1mGONZ1QkDf4AuTqYpCQDcawFsgOUJAtFn1udqaQkBEi2zn+7FCQEtZhjjW1UJAXkvIBz27QkAbL90kBsFCQEtZhjjW1UJAF9nO91PDQkCqYFRSJ9hCQN21hHzQi0JAvVKWIY69QkCwA+eMKP1CQPCnxks3MUNABOeMKO39Q0CmCkYldfpDQMHKoUW2A0RA8x/Sb19XQ0Ai/fZ14ERDQIV80LNZbUNAfa62Yn+pQ0AZBFYOLeJDQDqSy39ID0RA5IOezaovREBhw9MrZWFEQGHD0ytlYURAxyk6kstHRECjkjoBTWREQLyWkA96PkRAlPYGX5hUREBEi2zn+4FEQA5Pr5Rl6ERAQ61p3nEaRUBSSZ2AJgpFQMdLN4lBIEVAs3vysFALRUBqTfOOUwRFQMl2vp8aH0VAVn2utmIvRUA3GsBbIBFFQBKDwMqhzURAEoPAyqHNREDA7J48LLxEQD9XW7G/nERAlrIMcawjQ0DEsS5uozFDQOtztRX7c0NASnuDL0x2Q0D0/dR46RZEQLyWkA96PkRAK4cW2c7HREB0tRX7y85EQDcawFsgEUVAKH6MuWshRUDmriXkgz5FQCV1ApoIe0VAh6dXyjJ8RUAnoImw4blFQMNkqmBUEkZAr5RliGMdRkACK4cW2Q5GQPjCZKpgJEZArIvbaABXRkB1kxgEVuZFQLdif9k9oUVAQYLix5jTRUA+6Nms+gxGQEp7gy9MhkVANKK0N/haRUBvEoPAyllFQNEi2/l+ikRAl/+QfvsyRUAxCKwcWpRFQEGC4seY00VAaW/whckERkAmUwWjkkpFQPKwUGuaz0RAgSbChqdfRUCKjuTyH6pEQIy5awn52ERAseHplbIURUCBlUOLbCdFQP7UeOkmqUJAK/aX3ZNnQkDZzvdT4yVDQKyt2F92T0NA7Q2+MJlyQ0C9UpYhjr1DQMsQx7q41UNAbqMBvAXyQ0AUrkfhehxEQMoyxLEuLkRAG55eKcs4RED/snvysBBEQIts5/upwUNAirDh6ZW6Q0COBvAWSHBDQATnjCjttUJAFNBE2PAEQ0CvJeSDnvVCQFD8GHPXCkNA3bWEfNBzQ0AyVTAqqZNDQMNkqmBUskNA+zpwzoiSQkAzMzMzM4NCQMUgsHJoAUNA3NeBc0YUQ0BSuB6F6xFDQDJ3LSEfNENAvw6cM6JEQ0Aydy0hHzRDQDQRNjy9mkNAAJF++zq4Q0Cq8dJNYghFQG+BBMWPwUVAGQRWDi36RUBUUiegiVhGQD9XW7G/TEZABoGVQ4tERUB3LSEf9ORFQDMzMzMzY0VAcF8HzhkhRkDImLuWkO9FQLfRAN4C4UVAXW3F/rLrRUCxv+yePMxFQEoMAiuHnkVASgwCK4eeRUDjpZvEIDBFQEYldQKa4ERAmSoYldQBRUDF/rJ78ghFQNO84xQdaUVAb4EExY9hREDxY8xdS9hEQMIXJlMFA0VAGsBbIEERRUBU46WbxFBFQBb7y+7Jc0VAYTJVMCqJRUCgibDh6Y1FQK7YX3ZP9kRAYHZPHhbqREB3vp8aLz1FQFvTvOMU/URAW7G/7J5MRUDZPXlYqKVFQBb7y+7Jc0VA1JrmHae4RUDuWkI+6DFFQN/gC5OpkkVARiV1ApqYRUAxmSoYlcxFQG3F/rJ7ikJAUWuad5wqQ0AaUdobfAlEQOLplbIMQURA+FPjpZuMQ0CWQ4ts5zNEQCfChqdXKkRAhxbZzvfzQ0AlBoGVQwtEQLIubqMBHERAO3DOiNJ2REDu68A5I1pEQHNoke18N0RAbVZ9rrYyREDVCWgibBhEQAyTqYJRYURAqvHSTWKAREC+MJkqGFVFQPrt68A5k0VATYQNT6+0RUBNhA1Pr7RFQAfOGVHau0VAKqkT0ES4RUCV1AloIuRFQO2ePCzU8kVA7Z48LNTyRUDQ1VbsL/tFQHlYqDXNY0hA78nDQq2BSECx4emVsqxIQMe6uI0GkEhAGlHaG3yxSEBqvHSTGDxJQA3gLZCgMElAiPTb14GrSEBos+pzteVIQD0s1JrmLUlAukkMAitHSUA/xty1hIRJQE3zjlN0bElACD2bVZ/7SUCdgCbChtdJQLUV+8vuOUlAayv2l927SUAxmSoYlZxJQD2bVZ+rDUpAETY8vVLWSUDHSzeJQVhKQNejcD0KF0lAYcPTK2WhSUDXo3A9CndIQAdfmEwVPEhAbef7qfESR0B0tRX7yxZIQO/Jw0KtOUhA6Gor9pcVSEDVCWgibGBJQOVhodY030ZAJLn8h/TLRkCad5yiI2FHQFJJnYAmukdAw9MrZRlaSECRD3o2q75IQBgmUwWjwkdAB/AWSFD0R0DEQq1p3hlIQDojSnuDr0hA0SLb+X7yR0BseHqlLItIQF8pyxDHykhAqaROQBPpSEBmZmZmZvZIQJ+rrdhfRklAArwFEhSPSUDarPpcbQ1JQHbgnBGlTUlAPE7RkVxuSUCGONbFbUxJQDJ3LSEf/ElAOpLLf0j3SUCV1AloItRJQPH0SlmGqElAp+hILv9xSUClLEMc6/pJQCnLEMe6EEpAIbByaJEVSkBSJ6CJsCFKQEdy+Q/pb0hAz/dT46VbR0DWVuwvu49HQIy5awn5aEdAUiegibCJR0CMSuoENIFHQCh+jLlrwUdAJ6CJsOEZSEAwTKYKRgVIQP5l9+Rh+UdAhA1Pr

In [20]:
p.segment(df.seqs[dec], df.high[dec], df.seqs[dec], df.low[dec], color='red')

bokeh.models.renderers.GlyphRenderer(
    id='5d5afc8f-73d3-4cc3-b6b9-849875174518',
    data_source=bokeh.models.sources.ColumnDataSource(
        id='547785be-9866-4673-b276-837daf239a75',
        callback=None,
        data={'y1': {'__ndarray__': 'ayv2l92DO0BYqDXNO848QD0s1Jrm7TxA+MJkqmAUPUDByqFFtuM8QEI+6NmsajxA0gDeAgmKPUBj7lpCPlg9QJLLf0i//TxAumsJ+aB3PkBt5/up8YI+QIj029eBwz1A63O1FfurPUAnoImw4Vk/QCJseHqlLD9A7nw/NV4qP0CxUGuadzw/QGTMXUvIRz9AppvEILCKQEBDHOviNoJAQDj4wmSqcEBAiPTb14GrQEB90LNZ9WFAQMDsnjwsVEBA+8vuycO6QEBSJ6CJsKFAQK7YX3ZPjkBAe4MvTKbqQEBL6gQ0Ee5AQLwFEhQ/pkBArthfdk+OQEC8lpAPek5AQKCJsOHpLUBA/kP67esQQEAf9GxWfUZAQBNhw9MrjUBA2ht8YTKFQECHp1fKMkxAQM6qz9VWFEBAVp+rrdivP0DXEvJBz4Y/QPfkYaHWBD9AN4lBYOUYQEDKVMGopA5AQJ88LNSa1j9AUkmdgCaiP0DqBDQRNkRAQJM6AU2EBUBA4lgXt9E4QECjkjoBTSRAQKfoSC7/4T9AjZduEoPIQED99nXgnMFAQBB6Nqs+r0BA0ZFc/kPSQECxv+yePARBQDj4wmSq6EBAzhlR2hvsQECz6nO1FeNAQGlv8IXJdEFAeqUsQxzzQUDo2az6XO1BQHnpJjEIxEFAZ9XnaiumQUB/2T15WHhCQIj029eBQ0JADeAtkKDIQUDOGVHaG9RBQFOWIY51iUFAYOXQItvZQUC+MJkqGLVBQMzuycNC1UFASZ2AJsIeQkBOYhBYOexBQKUsQxzrykFA097gC5OhQUD+ZffkYXlBQKpgVFInGEJAtoR80LPZQUAbL90kBgFCQFHaG3xhEkJAv30dOGf8QUC9UpYhjv1BQBsv3SQGYUJAnu+nxksvQkB/2T15WIhCQLRZ9bnaakJA+aBns+ozQkDqBDQRNqxCQNcS8kHPxkJAAG+BBMW3QkCsi9toANdCQNcS8kHPxkJAMZkqGJXMQkAX2c73U8NCQBE2PL1S/kJABOeMKO39Q0Boke18PyVEQMZtNIC3KERAINJvXwcGRED+Q/rt69hDQPMf0m9ft0NAkKD4MeaOQ0CfPCzUmnZDQGUZ4lgXP0NAUiegibBZQ0Bt5/up8WJDQOSDns2qb0NAqMZLN4lpREC+wRcmU31EQJkqGJXUeURALpCg+DFuREAawFsgQVlEQCQofoy5Y0RAVn2utmIvRUDaG3xhMvVEQKTfvg6cW0VAx0s3iUEgRUCTGARWDu1EQNCzWfW56kRAnYAmwobXREDkg57Nqr9EQGPuWkI+4ERA54wo7Q2mREBPHhZqTctEQJf/kH77+kNAPQrXo3DFQ0Doaiv2l81DQJf/kH77GkRAio7k8h+KREB0tRX7y85EQAdfmEwV9EVAarx0kxh0RkAB3gIJikdGQOjZrPpc7UVAMzMzMzPDRUDZX3ZPHq5FQEymCkYltUVAgLdAguLHRUA+6Nms+gxGQB+F61G43kVAmbuWkA8iRkBq3nGKjrRFQL7BFyZTXUVAiIVa07xLRUA9CtejcKVEQFdbsb/s/kVAxty1hHzgRUCQoPgx5p5FQGiz6nO1bUVAx7q4jQZARUCqglFJnShFQKFns+pz1URA2/l+arx0RUDMf0i/fTVFQIofY+5aEkVAO3DOiNLeREBYqDXNO8ZEQC0hH/RszkRAGy/dJAZpRUAJG55eKUNDQHgLJCh+1EJAryXkg551QkAQ6bevA8dCQPmgZ7Pq60JAOGdEaW9IQ0DAWyBB8UtDQKyt2F92T0NAWKg1zTvmQ0Dn+6nx0v1DQM6qz9VWPERAPZtVn6sdREDfT42XbtpDQBZqTfOOa0NAdk8eFmpdQ0Csi9toAEdDQGwJ+aBn00JAdnEbDeC1Q0CP5PIf0hdDQG3F/rJ72kJAHVpkO9/PQkCoNc07TtlCQD2bVZ+rrUJAMnctIR98QkDG3LWEfAhDQHE9Ctej6EJAUrgehesRQ0B0tRX7y/ZCQKrx0k1imENAzqrP1VZkRkAm5IOezTJGQCNKe4MvzEVAeekmMQg0RUAlBoGVQ9NFQFD8GHPX6kVAXdxGA3gjRkCDwMqhRbZFQME5I0p7a0VAGXPXEvL5RUA0orQ3+JpFQEoMAiuHnkVAg8DKoUW2RUDpJjEIrGxFQLUV+8vuQUVAkst/SL8tRUCUh4Va0xRFQJVliGNdZEVARUdy+Q9JRUCIhVrTvFNFQKvP1VbsZ0VAj+TyH9InRUDWxW00gB9FQNPe4AuTGUVAC7WmecfhREAK16NwPZJEQEaU9gZfYERAQfFjzF1zREACmggbnm5EQK36XG3FFkRA7C+7Jw9rREC0WfW52qJEQAkbnl4pq0RAMuauJeT7RECTqYJRScVEQI4G8BZIkEVAih9j7lqKRUDr4jYawAtGQM4ZUdobfEVARUdy+Q9JRUDJdr6fGo9FQGlv8IXJnEVAVcGopE5QRUAm5IOezRpEQF+YTBWM2kJAAwmKH2MeREBfB84ZUXJDQG40gLdAskNAqz5XW7EHREAZ4lgXtwFEQIj029eBk0RAJ6CJsOE5REBWfa62Yk9EQMzuycNCbURAbef7qfEqRECjAbwFEmxEQDSitDf4ckVAyeU/pN/mRUCEDU+vlP1FQDQRNjy9AklAcvkP6bfXSEDHuriNBpBIQLByaJHtNElAf9k9eVgASUAEVg4tst1IQL99HThn9EhAldQJaCLUSEDM7snDQoVIQK5H4XoU1khArkfhehRGSUAAAAAAAAhJQIofY+5aQklAQmDl0CI7SUDVCWgibGBJQHKKjuTy30lAZF3cRgNYSUC94xQdyZ1JQLUV+8vu2UlAYVRSJ6BhSUAWak3zjuNJQKmkTkATGUpAeekmMQhkSUBrmnecontJQLFQa5p3DEpACD2bVZ9zSkAlBoGVQwtKQIXrUbgeNUlAY+5aQj54SEDEQq1p3oFIQGaIY13cLkhAowG8BRIcSECASL99HahHQC0hH/RshkdATx4Wak2jR0Bos+pztUVIQLUV+8vu+UdA63O1FfubSUC8dJMYBL5IQHS1FfvLFkhAeekmMQgkSEA17zhFR0pHQAXFjzF3pUZABcWPMXdFR0ANcayL26hIQBb7y+7JY0hA/tR46SYhSECrPldbsS9IQO58PzVe+kdAYHZPHhaCSECamZmZmXlIQJtVn6ut8EdAHhZqTfM2SEBAE2HD00tJQMiYu5aQb0lA2c73U+M9SUDXEvJBz8ZJQG3n+6nxqklArrZif9m1SUAzxLEubrNJQBgmUwWjWkpAXrpJDAJDSkCY3ZOHhTpKQA+cM6K0B0hAvJaQD3ouSEAwuycPC+1HQL4wmSoYdUdAtRX7y+55R0AYldQJaKpHQHrHKTqSI0hA1QloImwoSEBiEFg5tABIQNcS8kHPvkdAJ8KGp1fSR0BtVn2utnpIQOkmMQistEhAAAAAAACYSEAFo5I6AdVIQIqw4emVikhAVcGopE6gR0AeFmpN865HQNcS8kHPvkdAOGdEaW/YR0BUdCSX/0hIQGkAb4EE5UZAyAc9m1VXSEAbnl4py1hJQJqZmZmZiUlA6pWyDHFcSUA17zhFRzpJQJ2AJsKGd0pA3SQGgZWTSkBKe4MvTL5KQH3Qs1n1wUpAWMoyxLHWSkC7uI0G8I5LQK1p3nGKdktAVn2utmJ3S0A730+Nl75LQG3n+6nxqktAk6mCUUndS0A3GsBbIKFLQFCNl24Se0tAZF3cR

In [21]:
# bk.show(p) 
# SHOWS CHART inside notebook

In [22]:
p.rect(x='seqs', y='mid', width=w, height='height', fill_color='green', line_color='green', source=sourceInc)

bokeh.models.renderers.GlyphRenderer(
    id='b9a1c599-f09d-4ca5-b140-70e7798e4d92',
    data_source=bokeh.models.sources.ColumnDataSource(
        id='425a7285-5963-4ff0-97ce-25f6cbcf3040',
        callback=None,
        data={'label': ['Sep 5, 13',
          'Sep 9, 13',
          'Sep 10, 13',
          'Sep 11, 13',
          'Sep 13, 13',
          'Sep 18, 13',
          'Sep 19, 13',
          'Sep 23, 13',
          'Sep 25, 13',
          'Sep 26, 13',
          'Sep 27, 13',
          'Sep 30, 13',
          'Oct 1, 13',
          'Oct 2, 13',
          'Oct 4, 13',
          'Oct 10, 13',
          'Oct 11, 13',
          'Oct 14, 13',
          'Oct 16, 13',
          'Oct 17, 13',
          'Oct 18, 13',
          'Oct 21, 13',
          'Oct 30, 13',
          'Nov 4, 13',
          'Nov 5, 13',
          'Nov 6, 13',
          'Nov 8, 13',
          'Nov 13, 13',
          'Nov 14, 13',
          'Nov 20, 13',
          'Nov 21, 13',
          'Nov 22, 13',
          'Nov 27, 13',
          'Nov 29, 13',
          'Dec 2, 13',
          'Dec 3, 13',
          'Dec 4, 13',
          'Dec 9, 13',
          'Dec 16, 13',
          'Dec 18, 13',
          'Dec 20, 13',
          'Dec 24, 13',
          'Dec 26, 13',
          'Dec 30, 13',
          'Dec 31, 13',
          'Jan 7, 14',
          'Jan 10, 14',
          'Jan 14, 14',
          'Jan 15, 14',
          'Jan 16, 14',
          'Jan 28, 14',
          'Jan 29, 14',
          'Jan 30, 14',
          'Jan 31, 14',
          'Feb 6, 14',
          'Feb 7, 14',
          'Feb 10, 14',
          'Feb 11, 14',
          'Feb 12, 14',
          'Feb 13, 14',
          'Feb 14, 14',
          'Feb 19, 14',
          'Feb 20, 14',
          'Feb 21, 14',
          'Feb 27, 14',
          'Feb 28, 14',
          'Mar 4, 14',
          'Mar 6, 14',
          'Mar 11, 14',
          'Mar 12, 14',
          'Mar 14, 14',
          'Mar 17, 14',
          'Mar 18, 14',
          'Mar 20, 14',
          'Mar 24, 14',
          'Mar 28, 14',
          'Mar 31, 14',
          'Apr 1, 14',
          'Apr 8, 14',
          'Apr 9, 14',
          'Apr 11, 14',
          'Apr 14, 14',
          'Apr 15, 14',
          'Apr 16, 14',
          'Apr 22, 14',
          'Apr 24, 14',
          'Apr 28, 14',
          'May 7, 14',
          'May 8, 14',
          'May 12, 14',
          'May 13, 14',
          'May 16, 14',
          'May 19, 14',
          'May 21, 14',
          'May 29, 14',
          'May 30, 14',
          'Jun 4, 14',
          'Jun 5, 14',
          'Jun 10, 14',
          'Jun 13, 14',
          'Jun 16, 14',
          'Jun 17, 14',
          'Jun 18, 14',
          'Jun 20, 14',
          'Jun 23, 14',
          'Jun 25, 14',
          'Jun 27, 14',
          'Jul 1, 14',
          'Jul 2, 14',
          'Jul 7, 14',
          'Jul 10, 14',
          'Jul 11, 14',
          'Jul 15, 14',
          'Jul 16, 14',
          'Jul 18, 14',
          'Jul 21, 14',
          'Jul 25, 14',
          'Aug 4, 14',
          'Aug 7, 14',
          'Aug 12, 14',
          'Aug 13, 14',
          'Aug 14, 14',
          'Aug 15, 14',
          'Aug 18, 14',
          'Aug 19, 14',
          'Aug 21, 14',
          'Aug 28, 14',
          'Aug 29, 14',
          'Sep 3, 14',
          'Sep 4, 14',
          'Sep 5, 14',
          'Sep 8, 14',
          'Sep 9, 14',
          'Sep 10, 14',
          'Sep 11, 14',
          'Sep 16, 14',
          'Sep 17, 14',
          'Sep 18, 14',
          'Sep 19, 14',
          'Sep 24, 14',
          'Sep 26, 14',
          'Sep 29, 14',
          'Oct 3, 14',
          'Oct 8, 14',
          'Oct 15, 14',
          'Oct 16, 14',
          'Oct 17, 14',
          'Oct 20, 14',
          'Oct 21, 14',
          'Oct 23, 14',
          'Oct 27, 14',
          'Oct 28, 14',
          'Oct 29, 14',
          'Oct 31, 14',
          'Nov 3, 14',
          'Nov 4, 14',
          'Nov 5, 14',
          'Nov 6, 14',
          'Nov 10, 14

In [23]:
p.rect(x='seqs', y='mid', width=w, height='height', fill_color='red', line_color='red', source=sourceDec)

bokeh.models.renderers.GlyphRenderer(
    id='c725c044-51cf-4452-a82c-6c736b6fa20b',
    data_source=bokeh.models.sources.ColumnDataSource(
        id='9f034466-4555-4303-bfee-b26bef8a962e',
        callback=None,
        data={'label': ['Sep 6, 13',
          'Sep 12, 13',
          'Sep 16, 13',
          'Sep 17, 13',
          'Sep 20, 13',
          'Sep 24, 13',
          'Oct 3, 13',
          'Oct 7, 13',
          'Oct 8, 13',
          'Oct 15, 13',
          'Oct 22, 13',
          'Oct 23, 13',
          'Oct 24, 13',
          'Oct 25, 13',
          'Oct 28, 13',
          'Oct 29, 13',
          'Oct 31, 13',
          'Nov 1, 13',
          'Nov 7, 13',
          'Nov 11, 13',
          'Nov 12, 13',
          'Nov 15, 13',
          'Nov 18, 13',
          'Nov 19, 13',
          'Nov 25, 13',
          'Nov 26, 13',
          'Dec 5, 13',
          'Dec 6, 13',
          'Dec 10, 13',
          'Dec 11, 13',
          'Dec 12, 13',
          'Dec 13, 13',
          'Dec 17, 13',
          'Dec 19, 13',
          'Dec 23, 13',
          'Dec 27, 13',
          'Jan 2, 14',
          'Jan 3, 14',
          'Jan 6, 14',
          'Jan 8, 14',
          'Jan 9, 14',
          'Jan 13, 14',
          'Jan 17, 14',
          'Jan 21, 14',
          'Jan 22, 14',
          'Jan 23, 14',
          'Jan 24, 14',
          'Jan 27, 14',
          'Feb 3, 14',
          'Feb 4, 14',
          'Feb 5, 14',
          'Feb 18, 14',
          'Feb 25, 14',
          'Feb 26, 14',
          'Mar 3, 14',
          'Mar 5, 14',
          'Mar 7, 14',
          'Mar 10, 14',
          'Mar 13, 14',
          'Mar 19, 14',
          'Mar 21, 14',
          'Mar 25, 14',
          'Mar 26, 14',
          'Mar 27, 14',
          'Apr 2, 14',
          'Apr 3, 14',
          'Apr 4, 14',
          'Apr 7, 14',
          'Apr 10, 14',
          'Apr 21, 14',
          'Apr 23, 14',
          'Apr 25, 14',
          'Apr 29, 14',
          'May 1, 14',
          'May 2, 14',
          'May 5, 14',
          'May 6, 14',
          'May 14, 14',
          'May 15, 14',
          'May 22, 14',
          'May 23, 14',
          'May 27, 14',
          'May 28, 14',
          'Jun 2, 14',
          'Jun 3, 14',
          'Jun 9, 14',
          'Jun 11, 14',
          'Jun 12, 14',
          'Jun 19, 14',
          'Jun 24, 14',
          'Jun 26, 14',
          'Jun 30, 14',
          'Jul 3, 14',
          'Jul 8, 14',
          'Jul 9, 14',
          'Jul 14, 14',
          'Jul 17, 14',
          'Jul 22, 14',
          'Jul 23, 14',
          'Jul 24, 14',
          'Jul 28, 14',
          'Jul 29, 14',
          'Jul 30, 14',
          'Jul 31, 14',
          'Aug 1, 14',
          'Aug 5, 14',
          'Aug 8, 14',
          'Aug 11, 14',
          'Aug 20, 14',
          'Aug 22, 14',
          'Aug 25, 14',
          'Aug 26, 14',
          'Aug 27, 14',
          'Sep 2, 14',
          'Sep 12, 14',
          'Sep 15, 14',
          'Sep 22, 14',
          'Sep 23, 14',
          'Sep 25, 14',
          'Sep 30, 14',
          'Oct 1, 14',
          'Oct 2, 14',
          'Oct 6, 14',
          'Oct 7, 14',
          'Oct 9, 14',
          'Oct 10, 14',
          'Oct 13, 14',
          'Oct 14, 14',
          'Oct 22, 14',
          'Oct 24, 14',
          'Oct 30, 14',
          'Nov 7, 14',
          'Nov 14, 14',
          'Nov 18, 14',
          'Nov 19, 14',
          'Nov 21, 14',
          'Nov 24, 14',
          'Nov 25, 14',
          'Nov 28, 14',
          'Dec 2, 14',
          'Dec 3, 14',
          'Dec 5, 14',
          'Dec 8, 14',
          'Dec 10, 14',
          'Dec 15, 14',
          'Dec 16, 14',
          'Dec 24, 14',
          'Dec 26, 14',
          'Dec 29, 14',
          'Dec 30, 14',
          'Dec 31, 14',
          'Jan 5, 15',
          'Jan 6, 15',
          'Jan 9, 15',
          'Jan 12, 15',
          'Jan 13, 15',
          'Jan 14, 15',
          'Jan 15, 15',
          'Jan 21, 15',
      

In [31]:
# How we'll save out to our file, basically a file.write. Use f string to give dynamic, title is the html doc
bk.save(p, './stocks_api_project/static/candle_stick.html', title='5yr_candlestick')

/Users/madelinepet/.local/share/virtualenvs/stocks_api_project-ZUhAC5B9/lib/python3.7/site-packages/bokeh/io/saving.py:123: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")


'/Users/madelinepet/codefellows/stocks_api_project/stocks_api_project/static/candle_stick.html'